# Install

In [1]:
%pip install -q "monai-weekly[tqdm, nibabel, gdown, ignite]"

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


# !python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline
!pip install einops
!python -c "import monai" || pip install 'monai[all]'

# Import libraries

In [2]:
import os
import warnings
import datetime
warnings.filterwarnings("ignore") # remove some scikit-image warnings

from torch.utils.tensorboard import SummaryWriter

import monai
# monai.config.print_config()

from monai.apps import DecathlonDataset
from monai.data import DataLoader, CacheDataset, decollate_batch
from monai.utils import first, set_determinism
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss, DiceCELoss
from monai.inferers import sliding_window_inference
from monai.networks.nets import UNETR
from monai.transforms import (
    LoadImage,
    LoadImageD,
    EnsureChannelFirstD,
    ScaleIntensityD,
    ToTensorD,
    Compose,
    AsDiscreteD,
    SpacingD,
    OrientationD,
    ResizeD,
    RandAffineD,
    AsDiscrete,
    AsDiscreted,
    EnsureTyped,
    EnsureType,
    LoadImageD,
    EnsureChannelFirstD,
    OrientationD,
    SpacingD,
    ScaleIntensityD,
    ResizeD,
    RandAffineD,
    RandFlipD,
    RandRotateD,
    RandZoomD,
    ToTensorD,
)

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv


from monai.utils.misc import set_determinism

set_determinism(seed = 42)

In [3]:
cuda = torch.cuda.is_available()

device = torch.device("cuda" if cuda else "cpu")
num_workers = 4 if cuda else 0

print('You are using gpu if true, cpu if false:', cuda)

You are using gpu if true, cpu if false: True


## Load training set

In [4]:
KEYS = ["image", "label"]

def get_dataset():# Set up transform
    transform = Compose([
        LoadImageD(keys = KEYS),
        EnsureChannelFirstD(keys = KEYS),
        OrientationD(KEYS, axcodes='RAS'),
        SpacingD(keys = KEYS,
                 pixdim = (1., 1., 1.),
                 mode = ('bilinear', 'nearest')),
        ScaleIntensityD(keys = "image"),
        ResizeD(KEYS,
                (128, 128, 64),
                mode=('trilinear', 'nearest')),
        RandAffineD(keys = KEYS,
                    spatial_size = (128, 128, 64),
                    rotate_range = (0, 0, np.pi/12),
                    scale_range = (0.1, 0.1, 0.1),
                    mode = ('bilinear', 'nearest'),
                    prob = 1.0),
        ToTensorD(KEYS),
    ])

    # Load data
    dataset = DecathlonDataset(root_dir = "./",
                               task = "Task06_Lung", section = "training",
                               transform = transform, download = False)

    train_loader = DataLoader(dataset, batch_size = 4, shuffle = True, num_workers = 4)
    return (train_loader, dataset)

## Load validation set

In [5]:

def get_validation_indexes():
    list_val_dataset = []
    csv_file = 'val_dataset_NEW_64slices.csv'

    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        # Create a CSV reader object
        reader = csv.reader((line.replace('\0', '') for line in file))
        
        # Iterate over each row in the CSV file
        for row in reader:
            # Append the row to the list
            list_val_dataset.append(row)
            
    list_val_dataset = [[int(num) for num in row] for row in list_val_dataset]
    return list_val_dataset

val_transform = Compose([
    LoadImageD(keys = KEYS),
    EnsureChannelFirstD(keys = KEYS),
    OrientationD(KEYS, axcodes='RAS'),
    ScaleIntensityD(keys = "image"),
])

# Load validation data
val_dataset = DecathlonDataset(root_dir = "./",
                           task = "Task06_Lung", section = "validation",
                           transform = val_transform, download = False)
list_val_dataset = get_validation_indexes()

for idx in range(len(val_dataset)):
    # Access individual elements by index
    sample = val_dataset[idx]
    
    # Access image and label from the sample
    image = sample["image"]
    label = sample["label"]
    
    # (1, 512, 512, <no_slices>)
    print(image.shape)
    print(label.shape)
    
    image_slices = list_val_dataset[idx]
    print("Index " + str(idx))
    print(image_slices)
    
    image = image[..., image_slices]
    label = label[..., image_slices]
    
    sample["image"] = image
    sample["label"] = label

# Apply the remaining tranforms
KEYS = ["image", "label"]
remaining_val_transform = Compose([
    SpacingD(keys = KEYS,
             pixdim = (1., 1., 1.),
             mode = ('bilinear', 'nearest')),
    ResizeD(KEYS,
            (128, 128, 64),
            mode=('trilinear', 'nearest')),
    ToTensorD(KEYS),
])


for idx in range(len(val_dataset)):
    # Access individual elements by index
    sample = val_dataset[idx]
    transformed_sample = remaining_val_transform(sample)
    
    sample["image"] = transformed_sample["image"]
    sample["label"] = transformed_sample["label"]
    
    image = sample["image"]
    label = sample["label"]
    
    
    print(image.shape)
    print(label.shape)

val_loader = DataLoader(val_dataset, batch_size = 1, shuffle = True, num_workers = 4)


Loading dataset: 100%|██████████| 12/12 [00:47<00:00,  3.95s/it]


(1, 512, 512, 369)
(1, 512, 512, 369)
Index 0
[259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322]
(1, 512, 512, 255)
(1, 512, 512, 255)
Index 1
[180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243]
(1, 512, 512, 288)
(1, 512, 512, 288)
Index 2
[110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,

# Training Modified

In [6]:
def get_train_indexes():
    list_dataset = []
    csv_file = 'dataset_NEW_64slices.csv'

    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader((line.replace('\0', '') for line in file))
        for row in reader:
            list_dataset.append(row)
            
    list_dataset = [[int(num) for num in row] for row in list_dataset]
    return list_dataset


In [7]:
def get_new_dataset():

    KEYS = ["image", "label"]
    transform = Compose([
        LoadImageD(keys = KEYS),
        EnsureChannelFirstD(keys = KEYS),
        OrientationD(KEYS, axcodes='RAS'),
        ScaleIntensityD(keys = "image"),
    ])

    dataset_modified_training = DecathlonDataset(root_dir = "./",
                               task = "Task06_Lung", section = "training", transform=transform,
                               download=False)
    list_dataset = get_train_indexes()
    for idx in range(len(dataset_modified_training)):
        sample = dataset_modified_training[idx]
        image = sample["image"]
        label = sample["label"]

        image_slices = list_dataset[idx]

        image = image[..., image_slices]
        label = label[..., image_slices]

        sample["image"] = image
        sample["label"] = label


    KEYS = ["image", "label"]
    remaining_transform = Compose([
        SpacingD(keys = KEYS,
                 pixdim = (1., 1., 1.),
                 mode = ('bilinear', 'nearest')),
        ResizeD(KEYS,
                (128, 128, 64),
                mode=('trilinear', 'nearest')),
        RandAffineD(keys = KEYS,
                    spatial_size = (128, 128, 64),
                    rotate_range = (0, 0, np.pi/12),
                    scale_range = (0.1, 0.1, 0.1),
                    mode = ('bilinear', 'nearest'),
                    prob = 1.0),
        ToTensorD(KEYS),
    ])


    for idx in range(len(dataset_modified_training)):
        sample = dataset_modified_training[idx]
        transformed_sample = remaining_transform(sample)

        sample["image"] = transformed_sample["image"]
        sample["label"] = transformed_sample["label"]

        image = sample["image"]
        label = sample["label"]


    modified_train_loader = DataLoader(dataset_modified_training, batch_size = 4, shuffle = True, num_workers = 4)
    return (modified_train_loader, dataset_modified_training)

# Model, Loss, Optimizer

In [12]:
device = torch.device("cuda:1")
def initialize_new_model():
    
    UNETR_metadata = dict(
        in_channels=1,
        out_channels=2,
        img_size=(128, 128, 64), 
        feature_size=16,
        hidden_size=768,
        mlp_dim=3072,
        num_heads=12,
        pos_embed='conv',
        norm_name='instance',
        conv_block=True,
        res_block=True,
        dropout_rate=0.0,
        spatial_dims=3,
    )

    model = UNETR(**UNETR_metadata).to(device)
    return model

dice_loss = DiceLoss(to_onehot_y=True, softmax=True)
cross_entropy_loss = DiceCELoss(to_onehot_y=True, softmax=True)

def loss_function(output, label):
    return dice_loss(output, label)

dice_metric = DiceMetric(include_background=False, reduction="mean")


In [17]:
def train_function(model, max_epochs, train_loader, optimizer_name, lrate, val_loader, dataset):
    if(optimizer_name=='AdamW'):
        optimizer = torch.optim.AdamW(model.parameters(), lr=lrate)
    if(optimizer_name=='Adam'):
        optimizer = torch.optim.Adam(model.parameters(), lr=lrate)
    if(optimizer_name=='Nadam'):
        optimizer = torch.optim.NAdam(model.parameters(), lr=lrate)
    if(optimizer_name=='SGD'):
        optimizer = torch.optim.SGD(model.parameters(), lr=lrate)
    val_interval = 1
    best_metric = -1
    best_metric_epoch = -1
    metric_values = []
    epoch_loss_values =[]
    lr = lrate
    post_pred = Compose([AsDiscrete(argmax = True, to_onehot = 2)])
    post_label = Compose([AsDiscrete(to_onehot = 2)])
    current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_name = f"lr_{lrate}_epoch_{max_epochs}_model_{optimizer_name}_day_{current_date}"
    save_model_name = f"lr_{lrate}_epoch_{max_epochs}_model_{optimizer_name}_day_{current_date}.pth"
    writer = SummaryWriter(log_dir=f"runs_combo_diceloss_2/{run_name}")
    for epoch in range(max_epochs):
      print("-" * 12)
      print(f"Epoch {epoch + 1}/{max_epochs}")
      # Turn model to "train" mode
      model.train()
      epoch_loss = 0
      step = 0
      for batch_data in train_loader:
        step += 1

        input, label = (
            batch_data["image"].to(device),
            batch_data["label"].to(device),
        )

        # A common pytorch Deep Learning format to train model
        optimizer.zero_grad()
        output = model(input)
        loss = loss_function(output, label)
        loss.backward() # Compute gradient
        optimizer.step() # Update model's parameters

        epoch_loss += loss.item()
        print(f"{step}/{len(dataset) // train_loader.batch_size}, "
              f"train_loss: {loss.item():.4f}")
        writer.add_scalar("Loss/train", loss.item(), epoch * len(train_loader) + step)


      epoch_loss /= step
      epoch_loss_values.append(epoch_loss)

      print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

      writer.add_scalar("Loss/epoch", epoch_loss, epoch)

      if (epoch + 1) % val_interval == 0:
        # Turn model to "eval" mode
        model.eval()

        with torch.no_grad():
          for val_data in val_loader:
            val_input, val_label = (
                val_data["image"].to(device),
                val_data["label"].to(device),
            )
            roi_size = (128, 128, 64)
            sw_batch_size = 1

            # Set AMP for MONAI validation
            val_output = sliding_window_inference(
                val_input, roi_size, sw_batch_size, model
            )
            val_output = [post_pred(i) for i in decollate_batch(val_output)]
            val_label = [post_label(i) for i in decollate_batch(val_label)]

            # Compute metric for current iteration
            dice_metric(y_pred = val_output, y = val_label)

          # Aggregate the final mean dice result
          metric = dice_metric.aggregate().item()

          # Reset the status for the next epoch
          dice_metric.reset()

          metric_values.append(metric)
          if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            print("saved new best metric model!!!")
            torch.save(model.state_dict(), f"{save_model_name}")
#             torch.save(model.state_dict(), "best_metric_model.pth")

          print(
              f"current epoch: {epoch + 1},"
              f" current mean dice: {metric:.4f},"
              f" best mean dice: {best_metric:.4f},"
              f" at epoch: {best_metric_epoch}"
          )
          writer.add_scalar("Dice/val", metric, epoch)

    print(
        f"train completed, best metric: {best_metric:.4f}"
        f" at epoch: {best_metric_epoch}"
    )
    writer.close()
    return model

In [18]:
# # Set up transform
# val_transform = Compose([
#     LoadImageD(keys = KEYS),
#     EnsureChannelFirstD(keys = KEYS),
#     OrientationD(KEYS, axcodes='RAS'),
#     SpacingD(keys = KEYS, pixdim = (1., 1., 1.), mode = ('bilinear', 'nearest')),
#     ScaleIntensityD(keys = "image"),
#     ResizeD(KEYS, (128, 128, 64), mode=('trilinear', 'nearest')),
#     ToTensorD(KEYS),
# ])

# # Load data
# val_dataset_normal = DecathlonDataset(root_dir = "./",
#                            task = "Task06_Lung", section = "validation",
#                            transform = val_transform, download = False)

# val_loadernormal = DataLoader(val_dataset_normal, batch_size = 1, shuffle = True, num_workers = 4)
# train_loader, dataset = get_dataset()

# Start training

In [21]:
epoch_array = [300]
lr_array = [0.001]
optimizer_array =['Nadam']


epoch_array_modif = [300]
lr_array_modif = [0.001]
optimizer_array_modif = ['AdamW']

for i in range(len(epoch_array)):
    for j in range(len(optimizer_array)):
        for k in range(len(lr_array)):
            for m in range(len(epoch_array_modif)):
                for n in range(len(optimizer_array_modif)):
                    for p in range(len(lr_array_modif)):
                        print("Epochs ", epoch_array[i])
                        print("Optimizer ", optimizer_array[j])
                        print("Lr Rate ", lr_array[k])
                        model = initialize_new_model()
                        train_loader, dataset = get_dataset()
                        model = train_function(model, epoch_array[i], train_loader, optimizer_array[j], lr_array[k], 
                                               val_loader, dataset)
                        print("Epochs Modif ", epoch_array_modif[m])
                        print("Optimizer Modif ", optimizer_array_modif[n])
                        print("Lr Rate Modif ", lr_array_modif[p])
                        train_loader, dataset = get_new_dataset()
                        model = train_function(model, epoch_array_modif[m], train_loader, optimizer_array_modif[n], 
                                               lr_array_modif[p], val_loader, dataset)
                       # combo += 1

# train_loader, dataset = get_new_dataset()
# for i in range(len(epoch_array)):
#     for j in range(len(optimizer_array)):
#         for k in range(len(lr_array)):
#             print("Epochs ", epoch_array[i])
#             print("Optimizer ", optimizer_array[j])
#             print("Lr Rate ", lr_array[k])
#             model = initialize_new_model()
#             model = train_function(model, epoch_array[i], train_loader, optimizer_array[j], lr_array[k], 
#                                    val_loader, dataset)

        
          

Epochs  300
Optimizer  Nadam
Lr Rate  0.001


Loading dataset: 100%|██████████| 51/51 [06:51<00:00,  8.07s/it]

------------
Epoch 1/300


1/12, train_loss: 0.6669
2/12, train_loss: 0.6405
3/12, train_loss: 0.6222
4/12, train_loss: 0.6102
5/12, train_loss: 0.6017
6/12, train_loss: 0.6126
7/12, train_loss: 0.5957
8/12, train_loss: 0.5922
9/12, train_loss: 0.5894
10/12, train_loss: 0.5866
11/12, train_loss: 0.5846
12/12, train_loss: 0.5828
13/12, train_loss: 0.5829
epoch 1 average loss: 0.6052
saved new best metric model!!!
current epoch: 1, current mean dice: 0.0010, best mean dice: 0.0010, at epoch: 1
------------
Epoch 2/300
1/12, train_loss: 0.5823
2/12, train_loss: 0.5790
3/12, train_loss: 0.5772
4/12, train_loss: 0.5760
5/12, train_loss: 0.5754
6/12, train_loss: 0.5731
7/12, train_loss: 0.5714
8/12, train_loss: 0.5871
9/12, train_loss: 0.5699
10/12, train_loss: 0.5687
11/12, train_loss: 0.5666
12/12, train_loss: 0.5666
13/12, train_loss: 0.5651
epoch 2 average loss: 0.5737
current epoch: 2, current mean dice: 0.0000, best mean dice: 0.0010, at epoch: 1
------------
Epoch 3/300
1/12, train_loss: 0.5650
2/12, train_loss

11/12, train_loss: 0.5073
12/12, train_loss: 0.5073
13/12, train_loss: 0.5060
epoch 18 average loss: 0.5079
current epoch: 18, current mean dice: 0.0000, best mean dice: 0.0010, at epoch: 1
------------
Epoch 19/300
1/12, train_loss: 0.5079
2/12, train_loss: 0.5071
3/12, train_loss: 0.5073
4/12, train_loss: 0.5075
5/12, train_loss: 0.5075
6/12, train_loss: 0.5065
7/12, train_loss: 0.5075
8/12, train_loss: 0.5078
9/12, train_loss: 0.5075
10/12, train_loss: 0.5073
11/12, train_loss: 0.5076
12/12, train_loss: 0.5071
13/12, train_loss: 0.5068
epoch 19 average loss: 0.5073
current epoch: 19, current mean dice: 0.0000, best mean dice: 0.0010, at epoch: 1
------------
Epoch 20/300
1/12, train_loss: 0.5070
2/12, train_loss: 0.5073
3/12, train_loss: 0.5075
4/12, train_loss: 0.5073
5/12, train_loss: 0.5070
6/12, train_loss: 0.5067
7/12, train_loss: 0.5067
8/12, train_loss: 0.5067
9/12, train_loss: 0.5067
10/12, train_loss: 0.5065
11/12, train_loss: 0.5065
12/12, train_loss: 0.5064
13/12, train_l

...

1/12, train_loss: 0.1352
2/12, train_loss: 0.0715
3/12, train_loss: 0.1028
4/12, train_loss: 0.2562
5/12, train_loss: 0.1573
6/12, train_loss: 0.2180
7/12, train_loss: 0.0672
8/12, train_loss: 0.1291
9/12, train_loss: 0.1152
10/12, train_loss: 0.1976
11/12, train_loss: 0.1268
12/12, train_loss: 0.1184
13/12, train_loss: 0.0616
epoch 227 average loss: 0.1351
current epoch: 227, current mean dice: 0.4646, best mean dice: 0.5392, at epoch: 208
------------
Epoch 228/300
1/12, train_loss: 0.0626
2/12, train_loss: 0.1217
3/12, train_loss: 0.1961
4/12, train_loss: 0.1983
5/12, train_loss: 0.1432
6/12, train_loss: 0.0915
7/12, train_loss: 0.0935
8/12, train_loss: 0.2187
9/12, train_loss: 0.0510
10/12, train_loss: 0.0875
11/12, train_loss: 0.0823
12/12, train_loss: 0.1362
13/12, train_loss: 0.1846
epoch 228 average loss: 0.1282
current epoch: 228, current mean dice: 0.4903, best mean dice: 0.5392, at epoch: 208
------------
Epoch 229/300
1/12, train_loss: 0.0558
2/12, train_loss: 0.0668
3/12, 

8/12, train_loss: 0.0370
9/12, train_loss: 0.1916
10/12, train_loss: 0.0352
11/12, train_loss: 0.0523
12/12, train_loss: 0.0720
13/12, train_loss: 0.0675
epoch 244 average loss: 0.0900
current epoch: 244, current mean dice: 0.4449, best mean dice: 0.5392, at epoch: 208
------------
Epoch 245/300
1/12, train_loss: 0.0487
2/12, train_loss: 0.0855
3/12, train_loss: 0.1964
4/12, train_loss: 0.0632
5/12, train_loss: 0.0884
6/12, train_loss: 0.0488
7/12, train_loss: 0.0666
8/12, train_loss: 0.1760
9/12, train_loss: 0.0363
10/12, train_loss: 0.1294
11/12, train_loss: 0.0885
12/12, train_loss: 0.0487
13/12, train_loss: 0.0868
epoch 245 average loss: 0.0895
current epoch: 245, current mean dice: 0.4667, best mean dice: 0.5392, at epoch: 208
------------
Epoch 246/300
1/12, train_loss: 0.0786
2/12, train_loss: 0.0990
3/12, train_loss: 0.0805
4/12, train_loss: 0.0953
5/12, train_loss: 0.0172
6/12, train_loss: 0.0723
7/12, train_loss: 0.1663
8/12, train_loss: 0.0412
9/12, train_loss: 0.0606
10/12,

current epoch: 261, current mean dice: 0.4505, best mean dice: 0.5392, at epoch: 208
------------
Epoch 262/300
1/12, train_loss: 0.0483
2/12, train_loss: 0.1079
3/12, train_loss: 0.1127
4/12, train_loss: 0.1343
5/12, train_loss: 0.1530
6/12, train_loss: 0.0644
7/12, train_loss: 0.2006
8/12, train_loss: 0.0891
9/12, train_loss: 0.0775
10/12, train_loss: 0.0659
11/12, train_loss: 0.0808
12/12, train_loss: 0.0702
13/12, train_loss: 0.0545
epoch 262 average loss: 0.0969
current epoch: 262, current mean dice: 0.4526, best mean dice: 0.5392, at epoch: 208
------------
Epoch 263/300
1/12, train_loss: 0.1688
2/12, train_loss: 0.1343
3/12, train_loss: 0.1570
4/12, train_loss: 0.1076
5/12, train_loss: 0.0629
6/12, train_loss: 0.1544
7/12, train_loss: 0.0756
8/12, train_loss: 0.0938
9/12, train_loss: 0.0800
10/12, train_loss: 0.0289
11/12, train_loss: 0.0493
12/12, train_loss: 0.0355
13/12, train_loss: 0.0646
epoch 263 average loss: 0.0933
current epoch: 263, current mean dice: 0.4440, best mean

4/12, train_loss: 0.0511
5/12, train_loss: 0.0988
6/12, train_loss: 0.0581
7/12, train_loss: 0.1194
8/12, train_loss: 0.0536
9/12, train_loss: 0.0504
10/12, train_loss: 0.1518
11/12, train_loss: 0.0495
12/12, train_loss: 0.0494
13/12, train_loss: 0.2039
epoch 279 average loss: 0.0899
current epoch: 279, current mean dice: 0.4426, best mean dice: 0.5392, at epoch: 208
------------
Epoch 280/300
1/12, train_loss: 0.1717
2/12, train_loss: 0.0969
3/12, train_loss: 0.0575
4/12, train_loss: 0.0699
5/12, train_loss: 0.0414
6/12, train_loss: 0.0467
7/12, train_loss: 0.0570
8/12, train_loss: 0.0501
9/12, train_loss: 0.0591
10/12, train_loss: 0.1407
11/12, train_loss: 0.0791
12/12, train_loss: 0.1687
13/12, train_loss: 0.1023
epoch 280 average loss: 0.0878
current epoch: 280, current mean dice: 0.4890, best mean dice: 0.5392, at epoch: 208
------------
Epoch 281/300
1/12, train_loss: 0.0736
2/12, train_loss: 0.0246
3/12, train_loss: 0.0635
4/12, train_loss: 0.0579
5/12, train_loss: 0.0995
6/12, 

11/12, train_loss: 0.1090
12/12, train_loss: 0.0446
13/12, train_loss: 0.0501
epoch 296 average loss: 0.0914
current epoch: 296, current mean dice: 0.4486, best mean dice: 0.5392, at epoch: 208
------------
Epoch 297/300
1/12, train_loss: 0.0600
2/12, train_loss: 0.0756
3/12, train_loss: 0.2612
4/12, train_loss: 0.0727
5/12, train_loss: 0.0823
6/12, train_loss: 0.0616
7/12, train_loss: 0.0641
8/12, train_loss: 0.0969
9/12, train_loss: 0.1012
10/12, train_loss: 0.0374
11/12, train_loss: 0.0357
12/12, train_loss: 0.1229
13/12, train_loss: 0.0474
epoch 297 average loss: 0.0861
current epoch: 297, current mean dice: 0.4474, best mean dice: 0.5392, at epoch: 208
------------
Epoch 298/300
1/12, train_loss: 0.0422
2/12, train_loss: 0.2088
3/12, train_loss: 0.0408
4/12, train_loss: 0.0766
5/12, train_loss: 0.1191
6/12, train_loss: 0.0489
7/12, train_loss: 0.0789
8/12, train_loss: 0.1133
9/12, train_loss: 0.0587
10/12, train_loss: 0.0383
11/12, train_loss: 0.0397
12/12, train_loss: 0.0611
13/1

In [ ]:
plt.figure("train", (12, 6))
plt.subplot(1, 2, 1)
plt.title("Epoch Average Loss")
x = [i + 1 for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel("epoch")
plt.plot(x, y)
plt.subplot(1, 2, 2)
plt.title("Val Mean Dice")
x = [val_interval * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("epoch")
plt.plot(x, y)
plt.savefig('model_03_graph.png')
plt.show()
#plt.savefig(PATH_NAME+'model.png')

# Check model and Visualize

In [20]:
# print(
#     f"train completed, best_metric: {best_metric:.4f} "
#     f"at epoch: {best_metric_epoch}")
%load_ext tensorboard
%tensorboard --logdir=runs_combo_diceloss_2

In [ ]:
slide = 28
plt.figure("Test Model", (12, 12))

plt.subplot(1, 3, 1)
plt.title("Input")
plt.imshow(image[0, 0, :, :, slide], cmap = "gray")

plt.subplot(1, 3, 2)
plt.title("Label")
plt.imshow(image[0, 0, :, :, slide], cmap = "gray")
plt.imshow(label[0, 0, :, :, slide], cmap = 'jet', alpha = 0.5)

plt.show()

In [ ]:
model.cpu()
model.load_state_dict(torch.load("best_metric_model.pth"))

with torch.no_grad():
  output = model(image)
  output = torch.nn.Softmax()(output)
  output = torch.round(output)

  print(f"Output shape: {output.shape}")

  slide = 28
  plt.figure("Test Model", (12, 12))

  plt.subplot(1, 3, 1)
  plt.title("Input")
  plt.imshow(image[0, 0, :, :, slide], cmap = "gray")

  plt.subplot(1, 3, 2)
  plt.title("Label")
  plt.imshow(image[0, 0, :, :, slide], cmap = "gray")
  plt.imshow(label[0, 0, :, :, slide], cmap = 'jet', alpha = 0.5)

  plt.subplot(1, 3, 3)
  plt.title("Output")
  plt.imshow(image[0, 0, :, :, slide], cmap = "gray")
  plt.imshow(output[0, 1, :, :, slide], cmap = 'jet', alpha = 0.5)

  plt.show()

In [ ]:
def predict(image, label, model):
    with torch.no_grad():
      output = model(image)
      output = torch.nn.Softmax()(output)
      output = torch.round(output)

      print(f"Output shape: {output.shape}")

      slide = 28
      plt.figure("Test Model", (12, 12))

      plt.subplot(1, 3, 1)
      plt.title("Input")
      plt.imshow(image[0, 0, :, :, slide], cmap = "gray")

      plt.subplot(1, 3, 2)
      plt.title("Label")
      plt.imshow(image[0, 0, :, :, slide], cmap = "gray")
      plt.imshow(label[0, 0, :, :, slide], cmap = 'jet', alpha = 0.5)

      plt.subplot(1, 3, 3)
      plt.title("Output")
      plt.imshow(image[0, 0, :, :, slide], cmap = "gray")
      plt.imshow(output[0, 1, :, :, slide], cmap = 'jet', alpha = 0.5)

      plt.show()

for i in range(50):
    checker = first(val_loader)
    image, label = checker['image'], checker['label']
    predict(image, label, model)